#About this notebook

Initially, we prepare the dataframe with the stock data (all 75), which can be found [here](https://drive.google.com/file/d/1izaenCxi6-aVmnX4VKoP5wPhb34eu7i2/view?usp=sharing)

Then, we get the sentiment data from tweets with a specified keyword and run sentiment analysis on it using the Vader API. 

Maps sentiment onto stock data: 1min. interval. Example taken below is WMT and the other stocks we identified from the csv with P-values

Runs Logistic Regression on dataframe with and without sentiment, giving R-squared values for both which allows a comparison and thus lets us come to a conclusion whether or not sentiment improves overall accuracy (it does)

In [ ]:
!pip install alpha_vantage
!pip install vaderSentiment

In [ ]:
import statsmodels.tsa.stattools as ts
import tweepy
from tweepy import OAuthHandler
import time
import pytz
from alpha_vantage.timeseries import TimeSeries
import pandas as pd
import numpy as np
import re
import string
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer 

In [ ]:

def convert_datetime_timezone(dt, tz1, tz2):
    
    tz1 = pytz.timezone(tz1)
    tz2 = pytz.timezone(tz2)

    dt = datetime.datetime.strptime(dt,"%Y-%m-%d %H:%M:%S")
    dt = tz1.localize(dt)
    dt = dt.astimezone(tz2)
    dt = dt.strftime("%Y-%m-%d %H:%M:%S")

    return dt

#My key: 2ZJEXZ6IRC95SZHM
#ALPHA-VANTAGE INTRADAY STOCK DATA: 1MIN INTERVAL
bad_tickers_2 = []
stock_list=['MSFT','AAPL','AMZN','GOOGL','ORCL','JPM','GS','AXP','HSBC','V','BAC','T','JNJ','NVS','MRK','UNH','XOM','CVX','TOT','TM','F',
            'NSANY','TSLA','GM','HOG','BA','DAL','LUV','UAL','AAL','SAVE','MCD','CMG','KMX','MUSA','CTB','SAH','GPC','H','LVS','MAR','PFE','PG','CHL',
            'ABT','MDT','WMT','KO','PEP','BABA','NKE','DIS','NFLX','EROS','WWE','CNK','DELL','IBM','HPQ','FIT','HMI','TSM','USEG','PTR','SNP','UNP','SPCE',
            'LMT','NOC','UPS','FDX','RIO','VZ','SAP','UL']
			
ts1 = TimeSeries(key='EJ69MPM068NGTJ30', output_format='pandas')

UTC_TZ = 'Etc/UTC'
IST_TZ = 'Asia/Calcutta'

intraday_df=pd.DataFrame()

for stock in bad_tickers:
#for stock in bad_tickers:
  print(stock)
  try:
    data, meta_data = ts1.get_intraday(symbol=stock,interval='5min', outputsize='full')
    data.columns = ['Open', 'High', 'Low', 'Close', 'Volume']
    #data.set_index('Date', inplace = True)
    #RawTZ = pytz.timezone(meta_data['6. Time Zone'])
    #data['Date']=data.index
    #data.index = data['Date'].apply(convert_datetime_timezone, args=(RawTZ, UTC_TZ))	
    intraday_df[stock] = data['Close'].copy()
    time.sleep(15)
  
  except Exception as e:
    bad_tickers_2.append(stock)
    print('Exception in Slope: {c}, Message, {m}'.format(c = type(e).__name__, m = str(e)))
		
			
#https://gist.github.com/heyalexej/8bf688fd67d7199be4a1682b3eec7568

In [ ]:
intraday_df.head()

In [ ]:
intraday_df['Key']=intraday_df.index
df_1['Key']=df_1.index
Masterfin=pd.merge(df_1, intraday_df, on='Key')
from google.colab import drive

In [ ]:
Masterfin.index=df_1.index

In [ ]:
Masterfin

In [ ]:
Masterfin.drop('Key',axis=1,inplace=True)

In [ ]:
Masterfin.shape

In [ ]:
drive.mount('drive')
Masterfin.to_csv('intraday_75_NYSE.csv')
!cp intraday_75_NYSE.csv "drive/My Drive/"

In [ ]:
stock_list=['MSFT','AAPL','AMZN','GOOGL','ORCL','JPM','GS','AXP','HSBC','V','BAC','T','JNJ','NVS','MRK','UNH','XOM','CVX','TOT','TM','F',
            'NSANY','TSLA','GM','HOG','BA','DAL','LUV','UAL','AAL','SAVE','MCD','CMG','KMX','MUSA','CTB','SAH','GPC','H','LVS','MAR','PFE','PG','CHL',
            'ABT','MDT','WMT','KO','PEP','BABA','NKE','DIS','NFLX','EROS','WWE','CNK','DELL','IBM','HPQ','FIT','HMI','TSM','USEG','PTR','SNP','UNP','SPCE',
            'LMT','NOC','UPS','FDX','RIO','VZ','SAP','UL']
Masterfin1=pd.DataFrame()
stox1=[]
stox2=[]
min_pvalue=[]
best_lag=[]
for stock1 in stock_list:
  print(stock1)
  x1=Masterfin[stock1].tolist()
  for stock2 in stock_list:
    if (stock1==stock2):
      continue
    y1=Masterfin[stock2].tolist()
    data_=list(zip(x1,y1))
    try:
      granger_result=ts.grangercausalitytests(data_, maxlag=40, verbose=False)
    except Exception as e:
      print('Missing Data Error: '+str(e))
      print(': '+stock1)
      print(': '+stock2)
      continue
    pvalues=[]
    for i in range (40):
      ad=i+1
      a1=granger_result[ad]
      a1_=a1[0]
      x_params_1=a1_['params_ftest']
      x_likelihhod_ratio_1=a1_['lrtest']
      x_SSR_chi2_1=a1_['ssr_chi2test']
      pvalue=((x_params_1[1]+x_likelihhod_ratio_1[1]+x_SSR_chi2_1[1])/3) 
      pvalues.append(pvalue)
    min_pval=min(pvalues)
    min_index=0
    for i in range(len(pvalues)):
      if (pvalues[i]==min_pval):
        min_index=i+1
    min_pvalue.append(min_pval)
    best_lag.append(min_index)
    stox1.append(stock1)
    stox2.append(stock2)
Masterfin1['Stock1']=stox1
Masterfin1['Stock2']=stox2
Masterfin1['Min. p']=min_pvalue
Masterfin1['Best lag']=best_lag

In [ ]:
Masterfin1.to_csv('intraday_75_NYSE_pvalues.csv')
!cp intraday_75_NYSE_pvalues.csv "drive/My Drive/"

In [ ]:
Masterfin

In [ ]:
consumer_key= 'BoSNF6BKUrGxY5XgOTvnOIXaz'
consumer_secret= 'VkNsq3jBtMHZYtlYEUZTT377yM1dHahzPNVbp8fmDDxDzEwE8C'
access_token= '1263072369280577536-Oqw2FpEPSXmwDffAGofAwV91L1A7Tz'
access_token_secret= 'H7qfOjemcbuusp1pO7p3oiSY026QvrgI4OA9KDo60dtb6'

In [ ]:
class TwitterClient(object): 
    def __init__(self): 
        #Initialization method. 
        try: 
            # create OAuthHandler object 
            auth = OAuthHandler(consumer_key, consumer_secret) 
            # set access token and secret 
            auth.set_access_token(access_token, access_token_secret) 
            # create tweepy API object to fetch tweets 
            # add hyper parameter 'proxy' if executing from behind proxy "proxy='http://172.22.218.218:8085'"
            self.api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
            
        except tweepy.TweepError as e:
            print(f"Error: Tweeter Authentication Failed - \n{str(e)}")

    def get_tweets(self, query, maxTweets = 1000):
        #Function to fetch tweets. 
        # empty list to store parsed tweets 
        tweets = [] 
        sinceId = None
        max_id = -1
        tweetCount = 0
        tweetsPerQry = 100

        while tweetCount < maxTweets:
            try:
                if (max_id <= 0):
                    if (not sinceId):
                        new_tweets = self.api.search(q=query, count=tweetsPerQry)
                    else:
                        new_tweets = self.api.search(q=query, count=tweetsPerQry,
                                                since_id=sinceId)
                else:
                    if (not sinceId):
                        new_tweets = self.api.search(q=query, count=tweetsPerQry,
                                                max_id=str(max_id - 1))
                    else:
                        new_tweets = self.api.search(q=query, count=tweetsPerQry,
                                                max_id=str(max_id - 1),
                                                since_id=sinceId)
                if not new_tweets:
                    print("No more tweets found")
                    break

                time.sleep(0.1)

                for tweet in new_tweets:
                    parsed_tweet = {} 
                    parsed_tweet['tweets'] = tweet.text 
                    parsed_tweet['Date'] = tweet.created_at

                    # appending parsed tweet to tweets list 
                    if tweet.retweet_count > 0: 
                        # if tweet has retweets, ensure that it is appended only once 
                        if parsed_tweet not in tweets: 
                            tweets.append(parsed_tweet) 
                    else: 
                        tweets.append(parsed_tweet) 
                        
                tweetCount += len(new_tweets)
                print("Downloaded {0} tweets".format(tweetCount))
                max_id = new_tweets[-1].id

            except tweepy.TweepError as e:
                # Just exit if any error
                print("Tweepy error : " + str(e))
                break
        
        return pd.DataFrame(tweets)

In [ ]:
#GETTING TWEETS WHICH CONTAIN WALMART
twitter_client = TwitterClient()

# calling function to get tweets
tweets_dfWMT = twitter_client.get_tweets('Walmart', maxTweets=35000)
print(f'tweets_df Shape - {tweets_dfWMT.shape}')
tweets_dfWMT.tail(10)

Downloaded 100 tweets
Downloaded 200 tweets
Downloaded 300 tweets
Downloaded 400 tweets
Downloaded 500 tweets
Downloaded 600 tweets
Downloaded 700 tweets
Downloaded 800 tweets
Downloaded 900 tweets
Downloaded 1000 tweets
Downloaded 1100 tweets
Downloaded 1200 tweets
Downloaded 1300 tweets
Downloaded 1400 tweets
Downloaded 1500 tweets
Downloaded 1600 tweets
Downloaded 1700 tweets
Downloaded 1800 tweets
Downloaded 1900 tweets
Downloaded 2000 tweets
Downloaded 2100 tweets
Downloaded 2200 tweets
Downloaded 2300 tweets
Downloaded 2400 tweets
Downloaded 2500 tweets
Downloaded 2600 tweets
Downloaded 2700 tweets
Downloaded 2800 tweets
Downloaded 2900 tweets
Downloaded 3000 tweets
Downloaded 3100 tweets
Downloaded 3200 tweets
Downloaded 3300 tweets
Downloaded 3400 tweets
Downloaded 3500 tweets
Downloaded 3600 tweets
Downloaded 3700 tweets
Downloaded 3800 tweets
Downloaded 3900 tweets
Downloaded 4000 tweets
Downloaded 4100 tweets
Downloaded 4200 tweets
Downloaded 4300 tweets
Downloaded 4400 twee

Rate limit reached. Sleeping for: 671


Downloaded 23600 tweets
Downloaded 23700 tweets
Downloaded 23800 tweets
Downloaded 23900 tweets
Downloaded 24000 tweets
Downloaded 24100 tweets
Downloaded 24200 tweets
Downloaded 24300 tweets
Downloaded 24400 tweets
Downloaded 24500 tweets
Downloaded 24600 tweets
Downloaded 24700 tweets
Downloaded 24800 tweets
Downloaded 24900 tweets
Downloaded 25000 tweets
tweets_df Shape - (22901, 2)


,tweets,Date
22891,RT @ProfShimada: 略奪に遭った大手スーパーが再建せず撤退の意向。シカゴで。多...,2020-06-15 13:17:12
22892,RT @MichaelSkolnik: CD’s. \nWallet. \nSkittles...,2020-06-15 13:17:12
22893,RT @DanielGStone: Y’all remember when the gove...,2020-06-15 13:17:11
22894,RT @traceretrace: Walmart's gross profits in 2...,2020-06-15 13:17:10
22895,RT @DanielGStone: Y’all remember when the gove...,2020-06-15 13:17:09
22896,RT @DanielGStone: Y’all remember when the gove...,2020-06-15 13:17:08
22897,RT @DanielGStone: Y’all remember when the gove...,2020-06-15 13:17:03
22898,RT @DanielGStone: Y’all remember when the gove...,2020-06-15 13:17:02
22899,RT @KingSwankk: @FarhaanRT Remember when right...,2020-06-15 13:17:02
22900,RT @DanielGStone: Y’all remember when the gove...,2020-06-15 13:17:01


In [ ]:
#CLEANING UP TWEETS FOR SENTIMENT ANALYSIS
def clean_tweet(tweet):
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t]) |(\w+:\/\/\S+)", " ", tweet).split()) 
def remove_punct(text):
    text  = "".join([char for char in text if char not in string.punctuation])
    text = re.sub('[0-9]+', '', text)
    return text
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)        
    return input_txt
def clean_tweets(lst):
    # remove twitter Return handles (RT @xxx:)
    lst = np.vectorize(remove_pattern)(lst, "RT @[\w]*:")
    # remove twitter handles (@xxx)
    lst = np.vectorize(remove_pattern)(lst, "@[\w]*")
    # remove URL links (httpxxx)
    lst = np.vectorize(remove_pattern)(lst, "https?://[A-Za-z0-9./]*")
    # remove special characters, numbers, punctuations (except for #)
    lst = np.core.defchararray.replace(lst, "[^a-zA-Z#]", " ")
    return lst
text_tweets=clean_tweets(tweets_dfWMT['tweets'].tolist())
text_tweets=text_tweets.tolist()
text_tweet=[]
for tweet in text_tweets:
  tweet__text=clean_tweet(tweet)
  tweet___text=remove_punct(tweet__text)

  #REMOVING VARIOUS SYMBOLS FROM TWEET TEXT
  emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
  qwerty=(emoji_pattern.sub(r'', tweet___text)) # no emoji

  #for key1 in keywords_ext:
    #if (qwerty.find(key1) != -1):
  text_tweet.append(qwerty)
tweets_dfWMT['Cleaned tweets']=text_tweet

In [ ]:
tweets_dfWMT.drop('tweets',axis=1,inplace=True)

In [ ]:
xy=tweets_dfWMT['Date'].tolist()
dates=[]
times=[]
for z in xy:
  dates.append(z.date())
  times.append(z.time())
tweets_dfWMT.drop('Date', axis=1, inplace=True)
tweets_dfWMT['Date']=dates
tweets_dfWMT['Time']=times

In [ ]:
#RUNNING SENTIMENT SCORE ANALYSIS: VADER 
analyser = SentimentIntensityAnalyzer()
scores = []
text_tweet=tweets_dfWMT['Cleaned tweets'].tolist()
def sentiment_analyzer_scores(text, engl=True):
    if engl:
        trans = text
    else:
        trans = translator.translate(text).text
    score = analyser.polarity_scores(trans)
    lb = score['compound']
    return lb
scores=[]
for x in text_tweet:
  sentiment_score = sentiment_analyzer_scores(x)
  scores.append(sentiment_score)
tweets_dfWMT['SentimentScore']=scores

In [ ]:
tweets_dfWMT

,Cleaned tweets,Date,Time,SentimentScore
0,We need to focus on grocery stores because the...,2020-06-15,19:06:17,0.0000
1,Mahan Walmart Tallahassee Fl I cannot make thi...,2020-06-15,19:06:15,0.7506
2,Y’all remember when the government gave you t...,2020-06-15,19:06:14,0.0000
3,I almost collapsed there in Walmart today when...,2020-06-15,19:06:13,0.2944
4,Y’all remember when the government gave you t...,2020-06-15,19:06:11,0.0000
...,...,...,...,...
22896,Y’all remember when the government gave you t...,2020-06-15,13:17:08,0.0000
22897,Y’all remember when the government gave you t...,2020-06-15,13:17:03,0.0000
22898,Y’all remember when the government gave you t...,2020-06-15,13:17:02,0.0000
22899,Remember when right wing terrorists shot up a ...,2020-06-15,13:17:02,-0.8591


In [ ]:
#ALPHA-VANTAGE INTRADAY STOCK DATA: 1MIN INTERVAL

# ts = TimeSeries(key='EJ69MPM068NGTJ30', output_format='pandas')
# data, meta_data = ts.get_intraday(symbol='ORCL',interval='1min', outputsize='full')
#data['4. close'].head(n=369).plot()
#Master_fin57['ORCL']=data['4. close'].head(n=369).tolist()

In [ ]:
#HERE, WE GET THE CLOSE PRICES FOR ALL THE STOCKS WE HAD IDENTIFIED TO BE PART OF THE CAUSAL CHAIN FOR WMT, INCLUDING WMT ITSELF, 
#AND STORE IT INTO A DATAFRAME
ts = TimeSeries(key='EJ69MPM068NGTJ30', output_format='pandas')

sts=['WMT', 'ORCL', 'SAP', 'JNJ', 'CHL']
new_df=pd.DataFrame()

for st in sts:
  data, meta_data = ts.get_intraday(symbol=st,interval='1min', outputsize='full')
  new_df[st]=data['4. close'].copy()
  time.sleep(15)
new_df

,WMT,ORCL,SAP,JNJ,CHL
date,,,,,
2020-06-15 15:48:00,117.720,53.140,130.0801,140.820,34.4650
2020-06-15 15:47:00,117.685,53.130,130.0600,140.880,34.4550
2020-06-15 15:46:00,117.750,53.140,130.0650,140.800,34.4538
2020-06-15 15:45:00,117.770,53.115,130.0500,140.770,34.4600
2020-06-15 15:44:00,117.740,53.115,130.1100,140.750,34.4600
...,...,...,...,...,...
2020-06-09 09:35:00,121.530,54.910,137.2000,148.430,35.6400
2020-06-09 09:34:00,121.560,54.865,137.4500,148.120,35.6400
2020-06-09 09:33:00,121.200,54.860,137.3300,147.830,35.5700


In [ ]:
#NOW WE BEGIN MAPPING THE TWEET SENTIMENT SCORES ONTO THE DATAFRAME WITH THE STOCK DATA
ab=new_df.index.tolist()
dates_1=[]
times_1=[]
for c in ab:
  dates_1.append(c.date())
  times_1.append(c.time())
new_df['Date']=dates_1
new_df['Time']=times_1

In [ ]:
stock_priceWMT3=new_df.copy()

In [ ]:
import datetime
tweets_dfWMT3=tweets_dfWMT3[tweets_dfWMT3['Date']==datetime.date(2020, 6, 15)]
bruh=[]
for i in range(len(tweets_dfWMT3)):
  bruh.append(i)
tweets_dfWMT3.index=bruh
tweets_dfWMT3

,Cleaned tweets,Date,Time,SentimentScore
0,We need to focus on grocery stores because the...,2020-06-15,19:06:17,0.0000
1,Mahan Walmart Tallahassee Fl I cannot make thi...,2020-06-15,19:06:15,0.7506
2,Y’all remember when the government gave you t...,2020-06-15,19:06:14,0.0000
3,I almost collapsed there in Walmart today when...,2020-06-15,19:06:13,0.2944
4,Y’all remember when the government gave you t...,2020-06-15,19:06:11,0.0000
...,...,...,...,...
22896,Y’all remember when the government gave you t...,2020-06-15,13:17:08,0.0000
22897,Y’all remember when the government gave you t...,2020-06-15,13:17:03,0.0000
22898,Y’all remember when the government gave you t...,2020-06-15,13:17:02,0.0000
22899,Remember when right wing terrorists shot up a ...,2020-06-15,13:17:02,-0.8591


In [ ]:
stock_priceWMT3=stock_priceWMT3[stock_priceWMT3['Date']==datetime.date(2020, 6, 15)]
stock_priceWMT3

,WMT,ORCL,SAP,JNJ,CHL,Date,Time
date,,,,,,,
2020-06-15 15:48:00,117.720,53.140,130.0801,140.82,34.4650,2020-06-15,15:48:00
2020-06-15 15:47:00,117.685,53.130,130.0600,140.88,34.4550,2020-06-15,15:47:00
2020-06-15 15:46:00,117.750,53.140,130.0650,140.80,34.4538,2020-06-15,15:46:00
2020-06-15 15:45:00,117.770,53.115,130.0500,140.77,34.4600,2020-06-15,15:45:00
2020-06-15 15:44:00,117.740,53.115,130.1100,140.75,34.4600,2020-06-15,15:44:00
...,...,...,...,...,...,...,...
2020-06-15 09:35:00,117.550,51.015,127.5750,139.77,34.2000,2020-06-15,09:35:00
2020-06-15 09:34:00,117.740,51.090,127.7100,139.89,34.2000,2020-06-15,09:34:00
2020-06-15 09:33:00,118.290,51.160,127.9800,140.77,34.2800,2020-06-15,09:33:00


In [ ]:
#CODE FOR MAPPING THE TWEET SENTIMENT SCORES ONTO THE DATAFRAME WITH THE STOCK DATA
bhg=[]
for i in range(len(stock_priceWMT3)):
  bhg.append(i)
stock_priceWMT3.index=bhg
import datetime
karen={}
tweet_timeline1=tweets_dfWMT3['Time'].tolist()
stock_timeline1=stock_priceWMT3['Time'].tolist()

from datetime import datetime
for x in range(len(stock_timeline1)):
  seconds_dif=[]
  a=str(stock_timeline1[x])
  for y in range(len(tweet_timeline1)):
    b=str(tweet_timeline1[y])
    FMT = '%H:%M:%S'
    t1delta = datetime.strptime(a, FMT) - datetime.strptime(b, FMT)
    secs=t1delta.total_seconds()
    seconds_dif.append(secs)
  karen[':'+a]=seconds_dif
  sass=pd.DataFrame(data=karen)
karen2={}
for column in sass.columns:
  sas1=sass[(sass[column]<=30)&(sass[column]>=-30)]
  ab=sas1.index.tolist()
  karen2['sig'+str(column)]=ab
indi=[]
abc={}
for x in range(len(tweets_dfWMT3)):
  indi.append(x)
tweets_dfWMT3.index=indi
asdf=list(karen2.keys())
senscores=tweets_dfWMT3['SentimentScore'].tolist()
for keye in asdf:
  bc=karen2[keye]
  sur=0
  for inr in bc:
    sur=sur+senscores[inr]
  try:
    final_sum=sur/len(bc)
  except ZeroDivisionError:
    final_sum=0.0
  abc['f:'+keye]=final_sum
asp=[]
for i in range(len(stock_priceWMT3)):
  asp.append(i)
bruh2=pd.DataFrame(data=abc, index=[0])
df2 = bruh2.transpose()
df2['Sentiment Score']=df2[0]
df2['Time']=df2.index
df2.index=asp
df2.drop(0, axis=1, inplace=True)

In [ ]:
Master_fin59=stock_priceWMT3
Master_fin59['sentiment scores']=df2['Sentiment Score'].tolist()

In [ ]:
Master_fin57.columns=['Date','Time','WMT','WMT_sentiment']

In [ ]:
Master_fin59

,WMT,ORCL,SAP,JNJ,CHL,Date,Time,sentiment scores
0,117.720,53.140,130.0801,140.82,34.4650,2020-06-15,15:48:00,-0.043107
1,117.685,53.130,130.0600,140.88,34.4550,2020-06-15,15:47:00,0.022826
2,117.750,53.140,130.0650,140.80,34.4538,2020-06-15,15:46:00,0.014059
3,117.770,53.115,130.0500,140.77,34.4600,2020-06-15,15:45:00,-0.073187
4,117.740,53.115,130.1100,140.75,34.4600,2020-06-15,15:44:00,-0.036681
...,...,...,...,...,...,...,...,...
373,117.550,51.015,127.5750,139.77,34.2000,2020-06-15,09:35:00,0.000000
374,117.740,51.090,127.7100,139.89,34.2000,2020-06-15,09:34:00,0.000000
375,118.290,51.160,127.9800,140.77,34.2800,2020-06-15,09:33:00,0.000000
376,118.400,51.140,127.8800,140.61,34.2800,2020-06-15,09:32:00,0.000000


In [ ]:
#THE FINAL DATAFRAME IS BELOW
Master_fin59.columns = ['WMT', 'ORCL', 'SAP', 'JNJ', 'CHL', 'Date', 'Time', 'WMT_sentiment scores']
Master_fin59.drop(['Date', 'Time'], axis=1, inplace=True)
Master_fin59

,WMT,ORCL,SAP,JNJ,CHL,WMT_sentiment scores
0,117.720,53.140,130.0801,140.82,34.4650,-0.043107
1,117.685,53.130,130.0600,140.88,34.4550,0.022826
2,117.750,53.140,130.0650,140.80,34.4538,0.014059
3,117.770,53.115,130.0500,140.77,34.4600,-0.073187
4,117.740,53.115,130.1100,140.75,34.4600,-0.036681
...,...,...,...,...,...,...
373,117.550,51.015,127.5750,139.77,34.2000,0.000000
374,117.740,51.090,127.7100,139.89,34.2000,0.000000
375,118.290,51.160,127.9800,140.77,34.2800,0.000000
376,118.400,51.140,127.8800,140.61,34.2800,0.000000


In [ ]:
#WE FIRST GET A DATAFRAME WITH ONLY STOCK DATA FOR OUR INITIAL LOGISTIC REGRESSION TEST
Master_fin60=Master_fin59.drop('WMT_sentiment scores', axis=1)
Master_fin60

,WMT,ORCL,SAP,JNJ,CHL
0,117.720,53.140,130.0801,140.82,34.4650
1,117.685,53.130,130.0600,140.88,34.4550
2,117.750,53.140,130.0650,140.80,34.4538
3,117.770,53.115,130.0500,140.77,34.4600
4,117.740,53.115,130.1100,140.75,34.4600
...,...,...,...,...,...
373,117.550,51.015,127.5750,139.77,34.2000
374,117.740,51.090,127.7100,139.89,34.2000
375,118.290,51.160,127.9800,140.77,34.2800
376,118.400,51.140,127.8800,140.61,34.2800


In [ ]:
Master_fin60.fillna(method='ffill', inplace=True)
df_stocks_data3=Master_fin60.copy()
# dfsh=[]
# for i in range(len(df_stocks_data3)):
#   dfsh.append(i)
# df_stocks_data3.index=dfsh
for stock in (list(df_stocks_data3.columns)):
  signal = np.where(df_stocks_data3[stock] >= df_stocks_data3[stock].shift(), 1, 0)
  signal=pd.Series(signal)
  if stock=='WMT': 
    df_stocks_data3[stock+'_0sig'] = signal
    continue
  else:
    for lag in range(30):
      df_stocks_data3[stock+'_'+str(lag)+'sig'] = signal.shift(lag)
df_stocks_data3.dropna(inplace=True)

In [ ]:
df_stocks_data3

,WMT,ORCL,SAP,JNJ,CHL,WMT_0sig,ORCL_0sig,ORCL_1sig,ORCL_2sig,ORCL_3sig,ORCL_4sig,ORCL_5sig,ORCL_6sig,ORCL_7sig,ORCL_8sig,ORCL_9sig,ORCL_10sig,ORCL_11sig,ORCL_12sig,ORCL_13sig,ORCL_14sig,ORCL_15sig,ORCL_16sig,ORCL_17sig,ORCL_18sig,ORCL_19sig,ORCL_20sig,ORCL_21sig,ORCL_22sig,ORCL_23sig,ORCL_24sig,ORCL_25sig,ORCL_26sig,ORCL_27sig,ORCL_28sig,ORCL_29sig,SAP_0sig,SAP_1sig,SAP_2sig,SAP_3sig,...,JNJ_20sig,JNJ_21sig,JNJ_22sig,JNJ_23sig,JNJ_24sig,JNJ_25sig,JNJ_26sig,JNJ_27sig,JNJ_28sig,JNJ_29sig,CHL_0sig,CHL_1sig,CHL_2sig,CHL_3sig,CHL_4sig,CHL_5sig,CHL_6sig,CHL_7sig,CHL_8sig,CHL_9sig,CHL_10sig,CHL_11sig,CHL_12sig,CHL_13sig,CHL_14sig,CHL_15sig,CHL_16sig,CHL_17sig,CHL_18sig,CHL_19sig,CHL_20sig,CHL_21sig,CHL_22sig,CHL_23sig,CHL_24sig,CHL_25sig,CHL_26sig,CHL_27sig,CHL_28sig,CHL_29sig
29,117.690,53.235,130.740,141.200,34.57,1,0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0,0.0,1.0,0.0,...,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0
30,117.710,53.265,130.770,141.256,34.57,1,1,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1,0.0,0.0,1.0,...,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0
31,117.810,53.325,130.800,141.300,34.60,1,1,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1,1.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0
32,117.830,53.335,130.840,141.359,34.60,1,1,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1,1.0,1.0,0.0,...,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0
33,117.765,53.325,130.750,141.280,34.59,0,0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0,1.0,1.0,1.0,...,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
373,117.550,51.015,127.575,139.770,34.20,1,0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
374,117.740,51.090,127.710,139.890,34.20,1,1,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
375,118.290,51.160,127.980,140.770,34.28,1,1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
376,118.400,51.140,127.880,140.610,34.28,1,0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1

In [ ]:
import statsmodels.api as sm

In [ ]:
df1=df_stocks_data3.copy()
target_ar=df1['WMT_0sig'].tolist()
df1.drop('WMT_0sig', axis=1, inplace=True)
df1.drop('WMT', axis=1, inplace=True)
df1.to_numpy
sm_model = sm.Logit(target_ar, df1)
result = sm_model.fit(method='bfgs')

         Current function value: 0.436495
         Iterations: 35
         Function evaluations: 42
         Gradient evaluations: 42


/usr/local/lib/python3.6/dist-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


In [ ]:
test_results2=result.summary()

In [ ]:
#LOGISTIC REGRESSION RESULTS WITH SENTIMENT: 47%
print(test_results1)

                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                  349
Model:                          Logit   Df Residuals:                      194
Method:                           MLE   Df Model:                          154
Date:                Mon, 15 Jun 2020   Pseudo R-squ.:                  0.4700
Time:                        20:37:47   Log-Likelihood:                -128.20
converged:                      False   LL-Null:                       -241.90
Covariance Type:            nonrobust   LLR p-value:                 0.0001105
                                 coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------
ORCL                          -0.6987      1.862     -0.375      0.707      -4.347       2.950
SAP                            0.2503      1.520      0.165      0.869      -2.729 

In [ ]:
#LOGISTIC REGRESSION RESULTS WITHOUT SENTIMENT: 37.02%
print(test_results2)

                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                  349
Model:                          Logit   Df Residuals:                      225
Method:                           MLE   Df Model:                          123
Date:                Mon, 15 Jun 2020   Pseudo R-squ.:                  0.3702
Time:                        20:38:43   Log-Likelihood:                -152.34
converged:                      False   LL-Null:                       -241.90
Covariance Type:            nonrobust   LLR p-value:                 0.0007227
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ORCL          -0.6411      1.348     -0.476      0.634      -3.283       2.001
SAP            0.0890      1.142      0.078      0.938      -2.149       2.327
JNJ            0.0986      0.677      0.146      0.8

In [ ]:
wmt4=pd.concat([Master_fin56,wmt3])
dax=[]
for i in range(len(wmt4)):
  dax.append(i)
wmt4.index=dax
wmt4

,Date,Time,Close,sentiment scores
0,2020-06-12,16:00:00,16.7400,0.070087
1,2020-06-12,15:55:00,16.6650,-0.032073
2,2020-06-12,15:50:00,16.7400,0.037230
3,2020-06-12,15:45:00,16.6250,-0.038100
4,2020-06-12,15:40:00,16.5050,0.016000
...,...,...,...,...
360,2020-06-08,09:55:00,20.0983,0.106520
361,2020-06-08,09:50:00,19.9309,0.000000
362,2020-06-08,09:45:00,19.8705,0.000000
363,2020-06-08,09:40:00,19.4138,0.340000


In [ ]:
drive.mount('drive')
Masterfin.to_csv('INTRADAYSTOXDATA.csv')
!cp INTRADAYSTOXDATA.csv "drive/My Drive/"

In [ ]:
aal4

,Date,Time,Close,sentiment scores
0,2020-06-12,16:00:00,16.7400,0.070087
1,2020-06-12,15:55:00,16.6650,-0.032073
2,2020-06-12,15:50:00,16.7400,0.037230
3,2020-06-12,15:45:00,16.6250,-0.038100
4,2020-06-12,15:40:00,16.5050,0.016000
...,...,...,...,...
360,2020-06-08,09:55:00,20.0983,0.106520
361,2020-06-08,09:50:00,19.9309,0.000000
362,2020-06-08,09:45:00,19.8705,0.000000
363,2020-06-08,09:40:00,19.4138,0.340000


In [ ]:
Masterfin

,MSFT,AAPL,AMZN,GOOGL,ORCL,JPM,GS,AXP,RIO,VZ,SAP,UL,HSBC,V,BAC,T,JNJ,NVS,MRK,UNH,XOM,CVX,TOT,TM,F,NSANY,TSLA,GM,HOG,BA,DAL,LUV,UAL,AAL,SAVE,MCD,CMG,KMX,MUSA,CTB,SAH,GPC,H,LVS,MAR,PFE,PG,CHL,ABT,MDT,WMT,KO,PEP,BABA,NKE,DIS,NFLX,EROS,WWE,CNK,DELL,IBM,HPQ,FIT,HMI,TSM,USEG,PTR,SNP,UNP,SPCE,LMT,NOC,UPS,FDX
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-06-12 16:00:00,187.7300,338.5300,2543.7600,1413.260,51.850,99.820,201.78,101.6800,57.50,56.510,130.130,54.770,23.980,192.1900,24.7700,30.490,142.0300,84.3400,76.2900,284.870,47.140,92.3900,39.5900,127.7301,6.4600,7.930,935.2000,27.950,25.080,189.5100,30.4300,35.8700,39.6600,16.7400,18.8100,189.010,992.9500,89.9900,109.60,26.170,28.795,85.560,55.880,49.900,92.2500,33.750,115.620,34.9300,88.9900,93.200,117.680,45.5900,128.9400,217.680,96.380,115.4750,418.0900,3.2300,44.48,15.6500,46.900,121.9300,16.2800,6.1100,10.45,56.000,6.1200,34.310,44.1300,166.0900,14.9900,381.880,317.570,101.4700,132.7300
2020-06-12 15:55:00,188.0500,339.1100,2549.8401,1417.220,51.830,99.870,201.18,101.4500,57.46,56.660,129.750,54.820,23.975,192.2400,24.7100,30.530,142.1800,84.2700,76.3800,284.490,47.155,92.4500,39.5800,127.9400,6.4400,7.850,935.7600,27.820,25.025,189.1195,30.4100,35.7261,39.4650,16.6650,18.6850,188.911,992.6300,89.9550,109.43,26.180,28.850,85.640,55.890,49.800,91.9633,33.715,115.880,34.8900,89.1500,93.095,117.855,45.6050,129.1800,217.460,96.460,115.5798,418.6900,3.2000,44.51,15.6650,47.000,121.6800,16.2700,6.1400,10.45,55.910,NaN,34.300,44.0300,166.2100,14.9650,380.770,317.680,101.4300,132.2430
2020-06-12 15:50:00,187.9800,339.6700,2549.9124,1408.250,51.885,99.770,200.76,101.3200,57.48,56.585,129.950,54.770,23.985,191.7314,24.7350,30.510,142.2500,84.3200,76.5000,284.020,47.170,92.2900,39.5650,128.0200,6.4700,7.860,936.4800,27.890,25.075,188.8000,30.3900,35.6700,39.1767,16.7400,18.6000,189.120,991.6000,90.0561,109.34,26.080,28.810,85.850,55.670,49.580,91.5800,33.680,115.970,34.8899,89.0600,93.125,117.980,45.6600,129.1000,217.460,96.575,115.4200,416.5501,3.2200,44.41,15.7089,46.980,121.6800,16.3300,6.0950,10.44,55.855,NaN,34.280,43.9800,166.2000,14.9100,380.645,317.300,101.4900,132.1650
2020-06-12 15:45:00,187.3000,338.4400,2544.0000,1400.470,51.630,99.170,199.97,100.8000,57.28,56.335,129.495,54.550,23.895,190.9700,24.5801,30.395,141.6800,84.1150,76.2000,282.595,46.960,91.9600,39.4300,127.6200,6.4450,7.820,931.4829,27.780,24.980,187.3600,30.2810,35.5550,39.0250,16.6250,18.5700,188.240,988.7100,89.6100,108.98,25.840,28.640,85.480,55.530,49.350,91.0918,33.540,115.530,34.8800,88.6100,92.800,117.500,45.4682,128.4500,217.130,96.130,114.8550,415.4700,3.1800,44.06,15.6250,46.815,121.2800,16.2750,6.1000,10.42,55.690,6.1200,34.230,43.9000,165.6600,14.8850,378.510,315.610,101.0900,131.5100
2020-06-12 15:40:00,187.0400,337.8876,2539.0000,1395.095,51.585,99.050,199.83,100.8300,57.24,56.205,129.220,54.495,23.850,190.5500,24.5600,30.335,141.4318,84.0100,76.1400,282.260,46.835,91.8800,39.4000,127.5201,6.4450,7.810,929.7277,27.750,24.950,187.1500,30.2550,35.4900,39.1010,16.5050,18.6343,188.050,986.4700,89.5300,108.59,25.805,28.650,85.370,55.600,49.220,90.8850,33.475,115.430,34.8400,88.4350,92.640,117.410,45.4000,128.2800,216.830,96.130,114.8400,415.0733,3.1700,43.99,15.6350,46.720,121.0300,16.2550,6.0850,10.38,55.650,NaN,34.220,43.9200,165.2500,14.8867,378.480,315.600,100.9777,131.0800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-05-22 09:55:00,184.2300,317.2000,2460.3950,1404.595,52.400,89.260,179.22,88.8200,51.66,53.810,116.040,50.500,23.295,191.0800,22.6150,29.595,144.8300,84.1550,76.3400,285.280,43.830,90.7900,35.7200,117.5499,5.6076,7.000,827.9126,25.670,23.850,138.0500,22.8000,29.0623,25.6100,9.8950,10.1900,184